In [1]:
import os

In [2]:
%pwd

'/Users/arash/ML_End_to_End_Pj/end-to-end-solar-dust-detection/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/arash/ML_End_to_End_Pj/end-to-end-solar-dust-detection'

In [5]:
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/Arash-keshavarz/end-to-end-solar-dust-detection.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "E2N_SOLAR_DUST_DETECTION"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "7e519921c34f85b619643ad109cfc5ae3ff1b01a"



In [6]:
import torch




In [7]:
model = torch.load("artifacts/training/model.pt")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    params_classes: int

In [9]:
from solar_dust_detection.constants import *
from solar_dust_detection.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= "artifacts/training/model.pt",
            training_data= "artifacts/data_ingestion/Detect_solar_dust",
            all_params = self.params,
            mlflow_uri= "https://dagshub.com/Arash-keshavarz/end-to-end-solar-dust-detection.mlflow",
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE,
            params_classes= self.params.CLASSES
        )
        return eval_config

In [11]:
import torch
import mlflow
import mlflow.pytorch
from urllib.parse import urlparse
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms, models
import torch.nn as nn
from solar_dust_detection import logger

from dotenv import load_dotenv
load_dotenv()

class MapDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.dataset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.dataset)

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

    def _valid_generator(self):
        # 1. Define Transforms
        # Standard ImageNet normalization
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])

        val_transforms = transforms.Compose([
            transforms.Resize(self.config.params_image_size[:-1]), # Resize to (224, 224)
            transforms.ToTensor(),
            normalize
        ])

        # 2. Load Data
        full_dataset = datasets.ImageFolder(root=self.config.training_data)
        
        # 3. Create Split
        torch.manual_seed(42) 
        
        val_size = int(len(full_dataset) * 0.30)
        train_size = len(full_dataset) - val_size
        
        _, val_subset = random_split(full_dataset, [train_size, val_size])

        # 4. Apply Transforms
        val_dataset = MapDataset(val_subset, val_transforms)

        # 5. Create Loader
        self.valid_loader = DataLoader(
            val_dataset, 
            batch_size=self.config.params_batch_size, 
            shuffle=False,
            num_workers=0 
        )

    def load_model(self, path: Path) -> nn.Module:

        model = models.resnet18(weights=None)
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, self.config.params_classes) 
        
        # Load weights
        checkpoint = torch.load(path, map_location=self.device)
        model.load_state_dict(checkpoint)
        
        model.to(self.device)
        model.eval() 
        return model

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        
        criterion = nn.CrossEntropyLoss()
        
        running_loss = 0.0
        correct = 0
        total = 0

        # No gradient needed for evaluation
        with torch.no_grad():
            for images, labels in self.valid_loader:
                images, labels = images.to(self.device), labels.to(self.device)

                outputs = self.model(images)
                loss = criterion(outputs, labels)

                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Calculate averages
        avg_loss = running_loss / len(self.valid_loader)
        avg_acc = correct / total
        
        self.score = [avg_loss, avg_acc]
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        logger.info(f"Scores saved: {scores}")

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            
            # Switch to mlflow.pytorch
            if tracking_url_type_store != "file":
                mlflow.pytorch.log_model(self.model, "model", registered_model_name="ResNet18Model")
            else:
                mlflow.pytorch.log_model(self.model, "model")

In [12]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(config=eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    logger.exception(e)
    raise e

[2026-01-25 09:56:13,078: INFO: common]: YAML file: config/config.yaml loaded successfully
[2026-01-25 09:56:13,081: INFO: common]: YAML file: params.yaml loaded successfully
[2026-01-25 09:56:13,082: INFO: common]: Created directory at: artifacts
[2026-01-25 09:56:26,780: INFO: common]: JSON file saved at: scores.json
[2026-01-25 09:56:26,786: INFO: 2345937116]: Scores saved: {'loss': 0.1699026048745509, 'accuracy': 0.9700520833333334}


/Users/arash/ML_End_to_End_Pj/end-to-end-solar-dust-detection/.venv/lib/python3.12/site-packages/PIL/Image.py:1034: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


[2026-01-25 09:56:27,652: ERROR: 1860819819]: API request to endpoint /api/2.0/mlflow/runs/create failed with error code 401 != 200. Response body: '=============== ATTENTION! ===============

To use authentication, you must first: 
    Get your default access token from: https://dagshub.com/user/settings/tokens
    OR 
    Set a password: https://dagshub.com/user/settings/password 
=========================================='
Traceback (most recent call last):
  File "/var/folders/lw/cgx8yj0d66qcfy5dzjjrp6640000gn/T/ipykernel_32872/1860819819.py", line 6, in <module>
    evaluation.log_into_mlflow()
  File "/var/folders/lw/cgx8yj0d66qcfy5dzjjrp6640000gn/T/ipykernel_32872/2345937116.py", line 119, in log_into_mlflow
    with mlflow.start_run():
         ^^^^^^^^^^^^^^^^^^
  File "/Users/arash/ML_End_to_End_Pj/end-to-end-solar-dust-detection/.venv/lib/python3.12/site-packages/mlflow/tracking/fluent.py", line 539, in start_run
    active_run_obj = client.create_run(
                     ^

MlflowException: API request to endpoint /api/2.0/mlflow/runs/create failed with error code 401 != 200. Response body: '=============== ATTENTION! ===============

To use authentication, you must first: 
    Get your default access token from: https://dagshub.com/user/settings/tokens
    OR 
    Set a password: https://dagshub.com/user/settings/password 
=========================================='